In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from llama_cpp import Llama
from llama_cpp.llama_speculative import LlamaPromptLookupDecoding, LlamaDraftModel
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.chat_models import init_chat_model
from openai import OpenAI


In [ ]:
OPENAI_API_KEY = "sk-your-key"
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


# llm = init_chat_model("ft:gpt-4o-mini-2024-07-18:tu-graz-hereditary:gutbrain-ie-finetune:B5qr9cGV", model_provider="openai")
llm = init_chat_model("gpt-4o-mini-2024-07-18", model_provider="openai")

In [3]:
model = Llama.from_pretrained(
    "NousResearch/Hermes-3-Llama-3.2-3B-GGUF",
    filename="*.Q8_0.gguf",
    n_gpu_layers=-1,
    n_ctx=8096,
    temperature=0.1,
)

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    yes
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 4090, compute capability 8.9, VMM: yes
llama_load_model_from_file: using device CUDA0 (NVIDIA GeForce RTX 4090) - 7802 MiB free
llama_model_loader: loaded meta data with 38 key-value pairs and 255 tensors from /home/bkantz/.cache/huggingface/hub/models--NousResearch--Hermes-3-Llama-3.2-3B-GGUF/snapshots/3cd927095d8cbab12c743f932aa63b6f7bbfa141/./Hermes-3-Llama-3.2-3B.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Hermes 3 Llama 3.2 3b Base Fft Chatml...
llama_mode

In [4]:
model_path = "quants/llama-3-2-1B-instruct-lora.gguf"
# model = Llama(
#     model_path,
#     n_gpu_layers=-1,
#     n_ctx=4096,
#     temperature=0.1,
#     # draft_model=LlamaPromptLookupDecoding(num_pred_tokens=10),
# )

In [6]:
from constrerl.annotator import (
    Annotator,
    Article,
    load_train,
    load_test,
    convert_to_enum_model,
    convert_to_string_model,
    article_to_enum_model,
    ExtendedEnumERLModel,
    ExtendedStringERLModel,
)
from constrerl.erl_schema import convert_to_output
from constrerl._annotator_best import AnnotatorBest

In [7]:
data_path = "data/articles/articles_test.json"
out_path = "data/results/dev_out.json"
annotator = Annotator(model=model, gen_tokens=2048, add_rag=True, reorder=False, top_k=5)
# annotator = Annotator(model=model, gen_tokens=2048)
eval_set = load_test(data_path)
few_shot_samples=10
# annotator.add_prompt_examples([a for a in eval_set.values()][0:few_shot_samples])

In [8]:
with open("grammar.gbnf", "w") as f:
    f.write(annotator.erl_grammar)
print(annotator.erl_grammar)

root ::= (" "| "\n") grammar-models
grammar-models ::= relations
relations ::= "{"  ws "\"relations\"" ": " relations-relations  ws "}"
relation ::= "{"  ws "\"link_type\"" ": " relation-link-type ","  ws "\"subject_text_span\"" ": " string ","  ws "\"subject_location\"" ": " relation-subject-location ","  ws "\"object_text_span\"" ": " string ","  ws "\"object_location\"" ": " relation-object-location  ws "}"
relation-link-type ::= "\"anatomical location | located in | human\"" | "\"anatomical location | located in | animal\"" | "\"bacteria | interact | bacteria\"" | "\"bacteria | interact | chemical\"" | "\"bacteria | interact | drug\"" | "\"bacteria | influence | DDF\"" | "\"bacteria | change expression | gene\"" | "\"bacteria | located in | human\"" | "\"bacteria | located in | animal\"" | "\"bacteria | part of | microbiome\"" | "\"chemical | located in | anatomical location\"" | "\"chemical | located in | human\"" | "\"chemical | located in | animal\"" | "\"chemical | interact | c

In [9]:
annotator.example_messages

[{'role': 'system',
  'content': 'You are annotating a medical scientific title and abstract. You return all relation between entities within the title and abstract as JSON. The returned data include the relation type and text and should cover all relations over all relevant entities occurring in the text.'}]

In [10]:
test_samples = 10
annotations = annotator.annotate(
    {id: article for id, article in list(eval_set.items())[few_shot_samples:few_shot_samples+test_samples]}
)

Annotating articles:   0%|          | 0/10 [00:00<?, ?it/s]llama_perf_context_print:        load time =     722.12 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  4052 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /  2047 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   51077.24 ms /  6099 tokens
Annotating articles:  10%|█         | 1/10 [00:51<07:41, 51.26s/it, id=28767318]Llama.generate: 58 prefix-match hit, remaining 4312 prompt tokens to eval


Error in article 28767318
[{'text': 'While parsing a string, we missed the closing quote, ignoring', 'context': 'F | target'}]
removing last element


llama_perf_context_print:        load time =     722.12 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  4312 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /  2047 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   38556.10 ms /  6359 tokens
Annotating articles:  20%|██        | 2/10 [01:29<05:50, 43.82s/it, id=33742731]Llama.generate: 58 prefix-match hit, remaining 2937 prompt tokens to eval


Error in article 33742731
[{'text': 'While parsing a string, we missed the closing quote, ignoring', 'context': '",\n      "'}]
removing last element


llama_perf_context_print:        load time =     722.12 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  2937 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   893 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   12192.79 ms /  3830 tokens
Annotating articles:  30%|███       | 3/10 [01:42<03:25, 29.39s/it, id=28899766]Llama.generate: 58 prefix-match hit, remaining 6679 prompt tokens to eval
llama_perf_context_print:        load time =     722.12 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  6679 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   589 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    8847.92 ms /  7268 tokens
Annotating articles:  40%|████      | 4/10 [01:51<02:07, 21.32s/it, id=36271

...

llama_perf_context_print:        load time =     722.12 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  4790 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /  2047 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   29927.86 ms /  6837 tokens
Annotating articles:  80%|████████  | 8/10 [03:51<00:56, 28.45s/it, id=36801431]Llama.generate: 58 prefix-match hit, remaining 2159 prompt tokens to eval


Error in article 36801431
 {
    "relations": [
      {
        "link_type": "DDF | change abundance | microbiome",
        "subject_text_span": "PPI use",
        "subject_location": "abstract",
        "object_text_span": "gut microbiota",
        "object_location": "abstract"
      },
      {
        "link_type": "DDF | target | human",
        "subject_text_span": "kidney transplant recipients",
        "subject_location": "abstract",
        "object_text_span": "participants",
        "object_location": "abstract"
      },
      {
        "link_type": "DDF | target | human",
        "subject_text_span": "kidney transplant recipients",
        "subject_location": "abstract",
        "object_text_span": "TransplantLines Biobank and Cohort Study",
        "object_location": "abstract"
      },
      {
        "link_type": "DDF | target | human",
        "subject_text_span": "kidney transplant recipients",
        "subject_location": "abstract",
        "object_text_span": "kidney tra

llama_perf_context_print:        load time =     722.12 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  2159 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /  2047 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   28504.60 ms /  4206 tokens
Annotating articles:  90%|█████████ | 9/10 [04:20<00:28, 28.48s/it, id=37663891]Llama.generate: 58 prefix-match hit, remaining 3292 prompt tokens to eval


Error in article 37663891
[{'text': 'While parsing a string, we missed the closing quote, ignoring', 'context': 'cation": "'}]
removing last element


llama_perf_context_print:        load time =     722.12 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  3292 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   680 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    9138.32 ms /  3972 tokens
Annotating articles: 100%|██████████| 10/10 [04:29<00:00, 26.95s/it, id=33087994]


In [11]:
annotations

{'28767318': Relations(relations=[Relation(link_type=<LinkType.microbiome_is_linked_to_DDF: 'microbiome | is linked to | DDF'>, subject_text_span='gut microbiome', subject_location=<LabelLocation.TITLE: 'title'>, object_text_span='anxiety-related disorders', object_location=<LabelLocation.TITLE: 'title'>), Relation(link_type=<LinkType.microbiome_is_linked_to_DDF: 'microbiome | is linked to | DDF'>, subject_text_span='gut microbiome', subject_location=<LabelLocation.TITLE: 'title'>, object_text_span='trauma-related disorders', object_location=<LabelLocation.TITLE: 'title'>), Relation(link_type=<LinkType.DDF_target_human: 'DDF | target | human'>, subject_text_span='gut microbiome', subject_location=<LabelLocation.TITLE: 'title'>, object_text_span='mental health', object_location=<LabelLocation.TITLE: 'title'>), Relation(link_type=<LinkType.DDF_target_human: 'DDF | target | human'>, subject_text_span='gut microbiome', subject_location=<LabelLocation.TITLE: 'title'>, object_text_span='anxi

In [12]:
output_erl_model={
    id: convert_to_output(article).model_dump()
    for id, article in list(annotations.items())
}

In [13]:
output_erl_model

{'28767318': {'metadata': None,
  'entities': [],
  'relations': None,
  'binary_tag_based_relations': [{'subject_label': 'microbiome',
    'object_label': 'DDF'},
   {'subject_label': 'microbiome', 'object_label': 'DDF'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 'DDF', 'object_label': 'human'},
   {'subject_label': 

In [16]:
eval_set

{'30099552': Metadata(title='Making Sense of … the Microbiome in Psychiatry.', author='Thomaz F S Bastiaanssen; Caitlin S M Cowan; Marcus J Claesson; Timothy G Dinan; John F Cryan', journal='The international journal of neuropsychopharmacology', year='2019', abstract="Microorganisms can be found almost anywhere, including in and on the human body. The collection of microorganisms associated with a certain location is called a microbiota, with its collective genetic material referred to as the microbiome. The largest population of microorganisms on the human body resides in the gastrointestinal tract; thus, it is not surprising that the most investigated human microbiome is the human gut microbiome. On average, the gut hosts microbes from more than 60 genera and contains more cells than the human body. The human gut microbiome has been shown to influence many aspects of host health, including more recently the brain.Several modes of interaction between the gut and the brain have been di

In [ ]:
def f1k_annotations(y_true: list[str], y_pred: list[str], k: int = None):
    rel_set = set(y_true)
    # print(rel_set)
    doc_set = set(y_pred[:k])
    tp = len(doc_set.intersection(rel_set))  # docs that are in both -relevant docs
    fp = len(
        doc_set.difference(rel_set)
    )  # docs that are not in relevant set - irrelevant docs (false positiv)
    fn = len(
        rel_set.difference(doc_set)
    )  # relevant docs that are not present in doc set - missing docs
    if tp == 0:
        return 0, 0, 0
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    return 2 * precision * recall / (precision + recall), precision, recall


f1k_annotations_scores: list[float] = []

for id, article in annotations.items():
    gt_article = article_to_enum_model(eval_set[id], ExtendedEnumERLModel)
    gt_article = convert_to_string_model(gt_article, ExtendedStringERLModel)
    # print(article.relations[0].model_dump_json())
    y_true = [str(r.link_type) for r in gt_article.relations]
    y_pred = [str(r.link_type) for r in article.relations]
    print(y_true)
    print(y_pred)
    scores = f1k_annotations(y_true, y_pred)
    print(scores)
    f1k_annotations_scores.append(scores[0])

np.mean(f1k_annotations_scores)

['LinkType.DDF_strike_anatomical_location', 'LinkType.dietary_supplement_influence_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.dietary_supplement_influence_DDF', 'LinkType.DDF_affect_DDF', 'LinkType.DDF_affect_DDF', 'LinkType.dietary_supplement_influence_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.DDF_strike_anatomical_location', 'LinkType.dietary_supplement_influence_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.DDF_affect_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.microbiome_is_linked_to_DDF']
['LinkType.microbiome_is_linked_to_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.microbiome_is_linked_to_DDF', 'LinkType.drug_change_effect_DDF', 'LinkType.drug_change_effect_DDF']
(0.3333333333333333, 0.5, 0.25)
['LinkType.chemical_influence

np.float64(0.2921717171717172)